In [0]:
!pip install -q pyyaml h5py

In [0]:
import os 
import tensorflow as tf
from tensorflow import keras


In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [12]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [13]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
24/32 [=====================>........] - ETA: 0s - loss: 1.2756 - accuracy: 0.6406
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 13ms/step - loss: 1.1443 - accuracy: 0.6760 - val_loss: 0.7261 - val_accuracy: 0.7760
Epoch 2/10
27/32 [========================>.....] - ETA: 0s - loss: 0.4281 - accuracy: 0.8819
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4143 - accuracy: 0.8850 - val_loss: 0.5317 - val_accuracy: 0.8370
Epoch 3/10
27/32 [========================>.....] - ETA: 0s - loss: 0.2616 - accuracy: 0.9329
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.2783 - accuracy: 0.9300 - val_loss: 0.4693 - val_accuracy: 0.8580
Epoch 4/10
26/32 [=======================>......] - ETA: 0s - loss: 0.2220 - accuracy: 0.9447
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 

In [14]:
ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [15]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))


32/32 - 0s - loss: 2.3794 - accuracy: 0.0500
Untrained model, accuracy:  5.00%


In [16]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))


32/32 - 0s - loss: 0.4093 - accuracy: 0.8650
Untrained model, accuracy: 86.50%


In [17]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [19]:
ls(checkpoint_dir)

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `ls -F --color (checkpoint_dir)'


In [20]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [21]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labesl, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4897 - accuracy: 0.8730
Restored model, accuracy: 87.30%


In [22]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4897 - accuracy: 0.8730
Restored model, accuracy: 87.30%


In [23]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1196 - accuracy: 0.6890
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4390 - accuracy: 0.8720
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2770 - accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2013 - accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1485 - accuracy: 0.9700
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [24]:
ls saved_model

my_model/


In [26]:
new_model = tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [27]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}".format(100*acc))
print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4247 - accuracy: 0.8630
Restored model, accuracy: 86.30
(1000, 10)


In [29]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1263 - accuracy: 0.6710
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4240 - accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2860 - accuracy: 0.9340
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2025 - accuracy: 0.9510
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1512 - accuracy: 0.9670


In [30]:
new_model = tf.keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [31]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4348 - accuracy: 0.8610
Restored model, accuracy: 86.10%
